In [2]:
import pyspark
import os
os.getcwd()

'/Users/priyankapatil'

In [3]:
import findspark
findspark.init('/Users/priyankapatil/anaconda3/lib/python3.7/site-packages/pyspark')
print ("Hey Priyanka")

Hey Priyanka


In [4]:
from pyspark.sql import Row
from pyspark.sql import functions
from pyspark.sql import SQLContext
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as func
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.sql.warehouse.dir", "file:///C:/temp").appName("SparkSQL").getOrCreate()

sc = SparkContext.getOrCreate(SparkConf().setMaster("local[*]"))
sqlContext = SQLContext(sc)
sc

<SparkContext master=local[*] appName=SparkSQL>

In [5]:
# Load the data stored in csv format as a DataFrame.
data = spark.read.load(r'/Users/priyankapatil/Desktop/Data/dataset_flight.csv',
                     format="csv", sep=",", inferSchema='true', header="true")

In [6]:
#create a new copy of the Spark dataframe
dfa = data

In [7]:
#Create Table for explatory analysis

data.createOrReplaceTempView("Airline")

In [8]:
dfr1 = spark.sql("SELECT OP_UNIQUE_CARRIER,CANCELLED from Airline where CANCELLED =0")

dfr1.count()

14350435

In [9]:
#Total number of flights cancelled in January

dfr1b = spark.sql("SELECT OP_UNIQUE_CARRIER,CANCELLED from Airline where CANCELLED =1")

dfr1b.count()

248385

In [10]:
# % of flights cancelled

(dfr1b.count()/dfr1.count())*100

1.7308534549649541

In [11]:
#Carrier with most cancellations during Nov 2018 -  January 2019 period

dfr1b.groupby('OP_UNIQUE_CARRIER').sum('CANCELLED').orderBy('sum(CANCELLED)', ascending=False).show()

+-----------------+--------------+
|OP_UNIQUE_CARRIER|sum(CANCELLED)|
+-----------------+--------------+
|               WN|       48198.0|
|               AA|       34320.0|
|               OO|       28218.0|
|               MQ|       22438.0|
|               OH|       19338.0|
|               YX|       18363.0|
|               YV|       12238.0|
|               EV|       11511.0|
|               9E|       10726.0|
|               B6|       10059.0|
|               UA|        9922.0|
|               NK|        5415.0|
|               DL|        5368.0|
|               AS|        5246.0|
|               F9|        4709.0|
|               G4|        1460.0|
|               VX|         433.0|
|               HA|         423.0|
+-----------------+--------------+



In [12]:
# Weekday with most cancellations

dfr2 = spark.sql("SELECT DAY_OF_WEEK, CANCELLED from Airline where CANCELLED =1")

t = {1:"Monday", 2: "Tuesday", 3:"Wednesday", 4: "Thursday", 5: "Friday", 6: "Saturday", 7: "Sunday"}

conv = udf(lambda x: t[x], StringType())
dfwd= dfr2.withColumn("Weekday", conv("DAY_OF_WEEK")).groupBy('Weekday').sum('CANCELLED').orderBy('sum(CANCELLED)', ascending=False).show()

+---------+--------------+
|  Weekday|sum(CANCELLED)|
+---------+--------------+
|Wednesday|       43201.0|
| Thursday|       39165.0|
|  Tuesday|       38429.0|
|   Monday|       36741.0|
|   Friday|       34756.0|
|   Sunday|       32851.0|
| Saturday|       23242.0|
+---------+--------------+



In [13]:
#Average delay of flights originating from Tampa, JFK, ORD, LAX

dfrd = spark.sql("SELECT ORIGIN, DEP_DELAY, CANCELLED from Airline where (ORIGIN='TPA' or ORIGIN='ORD' or ORIGIN='JFK' or ORIGIN='LAX') and CANCELLED=0")


dfrd.groupBy('ORIGIN').avg('DEP_DELAY').orderBy('avg(DEP_DELAY)', ascending= False).show()

+------+------------------+
|ORIGIN|    avg(DEP_DELAY)|
+------+------------------+
|   ORD|13.696719592272279|
|   JFK|12.594131990182614|
|   TPA|11.045266409578227|
|   LAX| 9.197468209285526|
+------+------------------+



In [15]:
dfa.show(5)

+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+----+
|YEAR|QUARTER|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|TAIL_NUM|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|_c30|
+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+---

In [17]:
test = spark.sql("SELECT OP_UNIQUE_CARRIER,DEP_DELAY from Airline where DEP_DELAY > 15")

test.show(20)

+-----------------+---------+
|OP_UNIQUE_CARRIER|DEP_DELAY|
+-----------------+---------+
|               UA|    309.0|
|               UA|     25.0|
|               UA|     82.0|
|               UA|     42.0|
|               UA|     25.0|
|               UA|     98.0|
|               UA|     41.0|
|               UA|     77.0|
|               UA|    636.0|
|               UA|    143.0|
|               UA|     50.0|
|               UA|    118.0|
|               UA|     29.0|
|               UA|     29.0|
|               UA|   1069.0|
|               UA|     24.0|
|               UA|     35.0|
|               UA|     51.0|
|               UA|     42.0|
|               UA|    132.0|
+-----------------+---------+
only showing top 20 rows



In [18]:
from pyspark.sql.functions import lit


In [19]:
Delay15 = dfa.withColumn("DEP_DEL15", lit(0.0))
Delay15.show()

+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+----+---------+
|YEAR|QUARTER|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|TAIL_NUM|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|_c30|DEP_DEL15|
+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+------

In [36]:
Delay15 = dfa.withColumn("DEP_DEL15",when(dfa.DEP_DELAY > 15,1).otherwise(0))
Delay15.show()

+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+----+---------+
|YEAR|QUARTER|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|TAIL_NUM|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|_c30|DEP_DEL15|
+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+------

In [37]:
test2 = spark.sql("SELECT DEP_DEL15 from Airline1 where DEP_DELAY > 15")

test2.show(50)

+---------+
|DEP_DEL15|
+---------+
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
|        1|
+---------+
only showing top 50 rows



In [38]:
Delay15.createOrReplaceTempView("Airline1")

In [39]:
test3 = spark.sql("SELECT DEP_DEL15 from Airline1 where DEP_DELAY < 15")

test3.show(50)

+---------+
|DEP_DEL15|
+---------+
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
|        0|
+---------+
only showing top 50 rows



In [40]:
# Origin city to Destination Delay
dfrod= spark.sql("SELECT concat(ORIGIN,'-' ,DEST) as Route, DEP_DEL15, DEP_DELAY, CANCELLED from Airline1 where CANCELLED=0")


#Display route with most delays and avg delay
dfrod.groupby('Route').agg({'DEP_DEL15': 'sum','DEP_DELAY': 'avg'}).orderBy('sum(DEP_DEL15)', ascending= False).show(10)

+-------+--------------+------------------+
|  Route|sum(DEP_DEL15)|    avg(DEP_DELAY)|
+-------+--------------+------------------+
|ORD-LGA|          7625|20.180804432443807|
|LAX-SFO|          6650|13.238669744020143|
|LGA-ORD|          6508|18.694583874477868|
|SFO-LAX|          5562| 10.58283858350213|
|LAX-JFK|          4777|11.626711382915763|
|BOS-LGA|          4566|16.692887764061773|
|MCO-EWR|          4475|27.083448526736994|
|SEA-SFO|          4443|14.466033801580334|
|LGA-BOS|          4392|15.519232747094518|
|ATL-LGA|          4384|14.850805372719002|
+-------+--------------+------------------+
only showing top 10 rows



In [82]:
t=dfa.groupBy("ORIGIN").count()
t.sort(desc("count")).show(30)


+------+------+
|ORIGIN| count|
+------+------+
|   ATL|781668|
|   ORD|671038|
|   DFW|583910|
|   DEN|488330|
|   CLT|468298|
|   LAX|440613|
|   IAH|352271|
|   PHX|347026|
|   SFO|345978|
|   LGA|340673|
|   LAS|323737|
|   MSP|318856|
|   DTW|318729|
|   BOS|297804|
|   SEA|284242|
|   MCO|280172|
|   EWR|278887|
|   DCA|270107|
|   JFK|255764|
|   PHL|235712|
|   SLC|228019|
|   BWI|211429|
|   FLL|195313|
|   SAN|183756|
|   MIA|177122|
|   MDW|170245|
|   BNA|156372|
|   TPA|150344|
|   DAL|144587|
|   IAD|137699|
+------+------+
only showing top 30 rows



In [54]:
test4.show()

+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+----+---------+
|YEAR|QUARTER|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|TAIL_NUM|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|_c30|DEP_DEL15|
+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+------

In [81]:
HighLowtraffic_column = dfa.withColumn("HighLowTrafficOrigin",
      expr("case when ORIGIN = 'ATL' then 1 " +
                       "when ORIGIN = 'ORD' then 1 " +
                       "else 0 end"))
HighLowtraffic_column.show(10)

NameError: name 'Airline1' is not defined

In [79]:
test5 = spark.sql("SELECT * from Airline2").show(10)


+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+----+--------------------+
|YEAR|QUARTER|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|TAIL_NUM|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|_c30|HighLowTrafficOrigin|
+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------

In [77]:
Delay15.createOrReplaceTempView("Airline2")

In [78]:
HighLowtraffic_column.createOrReplaceTempView("Airline2")

In [80]:
dfa.head(10)

[Row(YEAR=2018, QUARTER=2, MONTH=4, DAY_OF_MONTH=24, DAY_OF_WEEK=2, OP_UNIQUE_CARRIER='UA', TAIL_NUM='N18243', OP_CARRIER_FL_NUM=2055, ORIGIN='SFO', DEST='PDX', CRS_DEP_TIME=1200, DEP_TIME=1155, DEP_DELAY=-5.0, TAXI_OUT=37.0, TAXI_IN=4.0, CRS_ARR_TIME=1351, ARR_TIME=1351, ARR_DELAY=0.0, CANCELLED=0.0, CANCELLATION_CODE=None, DIVERTED=0.0, CRS_ELAPSED_TIME=111.0, ACTUAL_ELAPSED_TIME=116.0, AIR_TIME=75.0, DISTANCE=550.0, CARRIER_DELAY=None, WEATHER_DELAY=None, NAS_DELAY=None, SECURITY_DELAY=None, LATE_AIRCRAFT_DELAY=None, _c30=None),
 Row(YEAR=2018, QUARTER=2, MONTH=4, DAY_OF_MONTH=24, DAY_OF_WEEK=2, OP_UNIQUE_CARRIER='UA', TAIL_NUM='N18243', OP_CARRIER_FL_NUM=2054, ORIGIN='PDX', DEST='SFO', CRS_DEP_TIME=1442, DEP_TIME=1436, DEP_DELAY=-6.0, TAXI_OUT=11.0, TAXI_IN=9.0, CRS_ARR_TIME=1632, ARR_TIME=1621, ARR_DELAY=-11.0, CANCELLED=0.0, CANCELLATION_CODE=None, DIVERTED=0.0, CRS_ELAPSED_TIME=110.0, ACTUAL_ELAPSED_TIME=105.0, AIR_TIME=85.0, DISTANCE=550.0, CARRIER_DELAY=None, WEATHER_DELAY=Non

In [83]:
dfa1 = dfa.withColumn("DEP_DEL15",when(dfa.DEP_DELAY > 15,1).otherwise(0)).withColumn("HighLowTrafficOrigin",
      expr("case when ORIGIN = 'ATL' then 1 " +
                       "when ORIGIN = 'ORD' then 1 " +
                       "when ORIGIN = 'DFW' then 1 " +
                       "when ORIGIN = 'DEN' then 1 " +
                       "when ORIGIN = 'CLT' then 1 " +
                       "when ORIGIN = 'LAX' then 1 " +
                       "when ORIGIN = 'IAH' then 1 " +
                       "when ORIGIN = 'PHX' then 1 " +
                       "when ORIGIN = 'SFO' then 1 " +
                       "when ORIGIN = 'LGA' then 1 " +
                       "else 0 end"))
dfa1.show()

+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+----+---------+--------------------+
|YEAR|QUARTER|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|TAIL_NUM|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|_c30|DEP_DEL15|HighLowTrafficOrigin|
+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------

In [84]:
dfa1.createOrReplaceTempView("Airline2")

In [85]:
test5 = spark.sql("SELECT * from Airline2").show(10)

+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+----+---------+--------------------+
|YEAR|QUARTER|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|TAIL_NUM|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|_c30|DEP_DEL15|HighLowTrafficOrigin|
+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------

In [145]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
import pandas as pd
from sklearn.cluster import KMeans



#copy dataframe
dataset = dfa1
dfa = dfa1

In [146]:
#Remove cancelled flights
dataset = dataset.filter("CANCELLED = 0")
dataset.show(10)

+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------------+-------------------+--------+--------+-------------+-------------+---------+--------------+-------------------+----+---------+--------------------+
|YEAR|QUARTER|MONTH|DAY_OF_MONTH|DAY_OF_WEEK|OP_UNIQUE_CARRIER|TAIL_NUM|OP_CARRIER_FL_NUM|ORIGIN|DEST|CRS_DEP_TIME|DEP_TIME|DEP_DELAY|TAXI_OUT|TAXI_IN|CRS_ARR_TIME|ARR_TIME|ARR_DELAY|CANCELLED|CANCELLATION_CODE|DIVERTED|CRS_ELAPSED_TIME|ACTUAL_ELAPSED_TIME|AIR_TIME|DISTANCE|CARRIER_DELAY|WEATHER_DELAY|NAS_DELAY|SECURITY_DELAY|LATE_AIRCRAFT_DELAY|_c30|DEP_DEL15|HighLowTrafficOrigin|
+----+-------+-----+------------+-----------+-----------------+--------+-----------------+------+----+------------+--------+---------+--------+-------+------------+--------+---------+---------+-----------------+--------+----------

In [100]:
datasett.unpersist()

DataFrame[YEAR: int, QUARTER: int, MONTH: int, DAY_OF_MONTH: int, DAY_OF_WEEK: int, OP_UNIQUE_CARRIER: string, TAIL_NUM: string, OP_CARRIER_FL_NUM: int, ORIGIN: string, DEST: string, CRS_DEP_TIME: int, DEP_TIME: int, DEP_DELAY: double, TAXI_OUT: double, TAXI_IN: double, CRS_ARR_TIME: int, ARR_TIME: int, ARR_DELAY: double, CANCELLED: double, CANCELLATION_CODE: string, DIVERTED: double, CRS_ELAPSED_TIME: double, ACTUAL_ELAPSED_TIME: double, AIR_TIME: double, DISTANCE: double, CARRIER_DELAY: double, WEATHER_DELAY: double, NAS_DELAY: double, SECURITY_DELAY: double, LATE_AIRCRAFT_DELAY: double, _c30: string, DEP_DEL15: int, HighLowTrafficOrigin: int]